# Lesson 3 — OpenCV Ball Tracking + Push Into Goal

Goal:
- find a coloured ball (OpenCV HSV)
- steer to centre it
- push forward into the goal

Rules:
- reuse `moves.py` for movement
- this lesson is about vision + choices


In [ ]:
# --- Bootstrap (same in every notebook) ---
from pathlib import Path
import sys

def add_repo_root():
    here = Path.cwd().resolve()
    for p in [here] + list(here.parents):
        if (p / 'lessons').is_dir() and (p / 'common').is_dir():
            if str(p) not in sys.path:
                sys.path.insert(0, str(p))
            print('Repo root:', p)
            return p
    raise FileNotFoundError('Could not find repo root (needs lessons/ and common/)')

add_repo_root()

In [ ]:
import time, importlib
import cv2
import numpy as np
from lesson01.level_1 import moves

importlib.reload(moves)
print('✔ OpenCV + moves ready')

In [ ]:
# Camera setup
CAM_INDEX = 0
cap = cv2.VideoCapture(CAM_INDEX)
if not cap.isOpened():
    raise RuntimeError('Camera failed to open. Try CAM_INDEX=1.')

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)
print('✔ camera opened')

## Choose your target colour (HSV)

Start with a single ball colour and tune in the room.


In [ ]:
# Example: green-ish. You will tune this.
LOW  = np.array([35,  80,  40])
HIGH = np.array([85, 255, 255])

MIN_AREA = 350
DEADZONE_PX = 25

print('✔ HSV set')

In [ ]:
def find_ball(frame_bgr):
    hsv = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, LOW, HIGH)
    mask = cv2.erode(mask, None, iterations=1)
    mask = cv2.dilate(mask, None, iterations=2)

    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts:
        return None
    c = max(cnts, key=cv2.contourArea)
    area = float(cv2.contourArea(c))
    if area < MIN_AREA:
        return None
    M = cv2.moments(c)
    if M['m00'] == 0:
        return None
    cx = int(M['m10'] / M['m00'])
    cy = int(M['m01'] / M['m00'])
    return {'cx': cx, 'cy': cy, 'area': area, 'mask': mask}


## Chase and push

- If ball is left → strafe left
- If ball is right → strafe right
- If centred → forward push


In [ ]:
def run(seconds=12, show_window=False):
    end = time.time() + float(seconds)
    try:
        while time.time() < end:
            ok, frame = cap.read()
            if not ok:
                moves.stop_now(); continue

            h, w = frame.shape[:2]
            mid_x = w // 2
            ball = find_ball(frame)

            if not ball:
                moves.stop_now()
                time.sleep(0.03)
                continue

            err = ball['cx'] - mid_x
            print(f"cx={ball['cx']} err={err} area={ball['area']:.0f}")

            if show_window:
                cv2.circle(frame, (ball['cx'], ball['cy']), 8, (255,255,255), 2)
                cv2.line(frame, (mid_x,0), (mid_x,h), (255,255,255), 1)
                cv2.imshow('ball', frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

            if err < -DEADZONE_PX:
                moves.move_left(0.06, speed=30)
            elif err > DEADZONE_PX:
                moves.move_right(0.06, speed=30)
            else:
                moves.forward(0.10, speed=34)

            time.sleep(0.01)
    finally:
        moves.stop_now()
        if show_window:
            cv2.destroyAllWindows()

run(seconds=10, show_window=False)

In [ ]:
cap.release()
moves.stop_now()
print('✔ camera released')